In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.1 MB/s eta 0:00:00


In [4]:
!pip install jsonschema

In [5]:
import json
from groq import Groq
import os

In [6]:
#  SETUP
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [7]:
#  JSON SCHEMA FOR EXTRACTION
SCHEMA = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Full name of the person"},
        "email": {"type": "string", "description": "Email address"},
        "phone": {"type": "string", "description": "Phone number with country code if available"},
        "location": {"type": "string", "description": "City, State, or Country"},
        "age": {"type": "integer", "description": "Age in years"}
    },
    "required": ["name", "email", "phone", "location", "age"],
    "additionalProperties": False
}

In [8]:
#  FUNCTION DEFINITION FOR GROQ
function_def = {
    "name": "extract_user_info",
    "description": "Extract structured user information from conversation text",
    "parameters": SCHEMA
}


In [9]:
#  SAMPLE CHATS
chats = [
    "Hi, I'm Rajesh Kumar. My email is rajesh.kumar@email.com. I live in Bangalore and my phone is +91 9876543210. I'm 28 years old.",
    "Hello! My name is Priya Sharma. You can reach me at priya.sharma@gmail.com. I'm from Delhi, age 32. Call me at 91-8888777766.",
    "Hey there! I'm Amit Singh. Phone: +1 (555) 123-4567. Email: amit.singh@work.com. I'm 35 and based in San Francisco."
]

In [10]:
#  FUNCTION TO EXTRACT USING GROQ FUNCTION CALLING
def extract_with_groq(chat_text):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts structured user details from chat text."},
            {"role": "user", "content": chat_text}
        ],
        functions=[function_def],
        function_call={"name": "extract_user_info"}  # Force call this function
    )

    # Extract function call result
    func_call = response.choices[0].message.function_call
    if func_call and func_call.name == "extract_user_info":
        return json.loads(func_call.arguments)
    else:
        return {"error": "Function call failed or no valid extraction"}

In [13]:
#  VALIDATION FUNCTION
def validate_output(data, schema):
    """Validate extracted data against JSON schema"""
    try:
        from jsonschema import validate, ValidationError
        validate(instance=data, schema=schema)
        return True, "Valid"
    except ValidationError as e:
        return False, str(e)
    except ImportError:
        return None, "jsonschema library not installed (optional validation)"

In [14]:
# --- RUN FOR ALL CHATS ---
print("Structured Info Extraction via Groq Function Calling\n")

for i, chat in enumerate(chats, 1):
    print(f"Chat {i}")
    print(f"Input: {chat}\n")

    result = extract_with_groq(chat)
    print(f"Extracted: {json.dumps(result, indent=2)}\n")

    is_valid, msg = validate_output(result, SCHEMA)
    if is_valid is None:
        print(f" Validation skipped (install jsonschema for full validation)")
    elif is_valid:
        print(f" Validated: {msg}")
    else:
        print(f" Invalid: {msg}")

    print("-" * 60)

Structured Info Extraction via Groq Function Calling

Chat 1
Input: Hi, I'm Rajesh Kumar. My email is rajesh.kumar@email.com. I live in Bangalore and my phone is +91 9876543210. I'm 28 years old.

Extracted: {
  "age": 28,
  "email": "rajesh.kumar@email.com",
  "location": "Bangalore",
  "name": "Rajesh Kumar",
  "phone": "+91 9876543210"
}

 Validated: Valid
------------------------------------------------------------
Chat 2
Input: Hello! My name is Priya Sharma. You can reach me at priya.sharma@gmail.com. I'm from Delhi, age 32. Call me at 91-8888777766.

Extracted: {
  "age": 32,
  "email": "priya.sharma@gmail.com",
  "location": "Delhi",
  "name": "Priya Sharma",
  "phone": "91-8888777766"
}

 Validated: Valid
------------------------------------------------------------
Chat 3
Input: Hey there! I'm Amit Singh. Phone: +1 (555) 123-4567. Email: amit.singh@work.com. I'm 35 and based in San Francisco.

Extracted: {
  "age": 35,
  "email": "amit.singh@work.com",
  "location": "San Franc